# Data preprocessing

In [1]:
import pandas as pd
import random
import sys
sys.path.append('F:\\NLP_CV\\project\\1project\\SIF-master\\src')
import data_io, params, SIF_embedding
import jieba
import re
import numpy as np
import tensorflow as tf
from gensim.models.word2vec import Word2Vec,LineSentence
import os
from collections import defaultdict
from six.moves import cPickle as pickle
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score

e:\anaconda2\envs\tf\lib\site-packages\tensorflow\python\framework\dtypes.py:523: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
e:\anaconda2\envs\tf\lib\site-packages\tensorflow\python\framework\dtypes.py:524: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
e:\anaconda2\envs\tf\lib\site-packages\tensorflow\python\framework\dtypes.py:525: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
e:\anaconda2\envs\tf\lib\site-packages\tensorflow\python\framework\dtypes.py:526: FutureWarning: Passing (type, 1) or 

In [2]:
csv_file = './movie_comments.csv'

In [3]:
raw_df = pd.read_csv(csv_file)

e:\anaconda2\envs\tf\lib\site-packages\IPython\core\interactiveshell.py:3020: DtypeWarning: Columns (0,4) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
raw_df.head()

,id,link,name,comment,star
0,1,https://movie.douban.com/subject/26363254/,战狼2,吴京意淫到了脑残的地步，看了恶心想吐,1
1,2,https://movie.douban.com/subject/26363254/,战狼2,首映礼看的。太恐怖了这个电影，不讲道理的，完全就是吴京在实现他这个小粉红的英雄梦。各种装备轮...,2
2,3,https://movie.douban.com/subject/26363254/,战狼2,吴京的炒作水平不输冯小刚，但小刚至少不会用主旋律来炒作…吴京让人看了不舒服，为了主旋律而主旋...,2
3,4,https://movie.douban.com/subject/26363254/,战狼2,凭良心说，好看到不像《战狼1》的续集，完虐《湄公河行动》。,4
4,5,https://movie.douban.com/subject/26363254/,战狼2,中二得很,1


In [5]:
len(raw_df)

261497

In [6]:
set(raw_df['star'])

{1, '1', 2, '2', 3, '3', 4, '4', '5', 5, 'star'}

In [7]:
raw_df[raw_df['star'] == 'star']

,id,link,name,comment,star
568,id,link,name,comment,star


In [9]:
nonan_df = raw_df.dropna(subset=['comment', 'star'])

In [10]:
len(nonan_df)

261495

In [23]:
nonan_df['comment'][:10]

0                                   吴京意淫到了脑残的地步，看了恶心想吐
1    首映礼看的。太恐怖了这个电影，不讲道理的，完全就是吴京在实现他这个小粉红的英雄梦。各种装备轮...
2    吴京的炒作水平不输冯小刚，但小刚至少不会用主旋律来炒作…吴京让人看了不舒服，为了主旋律而主旋...
3                        凭良心说，好看到不像《战狼1》的续集，完虐《湄公河行动》。
4                                                 中二得很
5                          “犯我中华者，虽远必诛”，吴京比这句话还要意淫一百倍。
6                                    脑子是个好东西，希望编剧们都能有。
7    三星半，实打实的7分。第一集在爱国主旋律内部做着各种置换与较劲，但第二集才真正显露吴京的野心...
8    开篇长镜头惊险大气引人入胜 结合了水平不俗的快剪下实打实的真刀真枪 让人不禁热血沸腾 特别弹...
9    15/100吴京的冷峰在这部里即像成龙，又像杰森斯坦森，但体制外的同类型电影，主角总是代表个...
Name: comment, dtype: object

In [12]:
def cut(string):
    return jieba.lcut(string)

In [26]:
re_obj = re.compile('[\Wa-zA-Z]')

In [28]:
%time data_df = nonan_df.apply(lambda x: pd.Series([re_obj.sub('',x['comment']) if re_obj.sub('',x['comment']) != '' else None, int(x['star']) if x['star']!='star' else None] , index=['comment', 'label']), axis=1)

Wall time: 1min 6s


In [13]:
%time data_df = nonan_df.apply(lambda x: pd.Series([re.sub('[\Wa-zA-Z]','',x['comment']) if re.sub('[\Wa-zA-Z]','',x['comment']) != '' else None, int(x['star']) if x['star']!='star' else None] , index=['comment', 'label']), axis=1)

Wall time: 1min 5s


In [14]:
data_df.iloc[58]

comment    燃大场面真的不输国外大片不尴尬吴京打戏很精彩水下搏斗看着也很有力必须安利一下张翰这角色简直就...
label                                                      5
Name: 58, dtype: object

In [19]:
len(data_df)

261495

In [20]:
data_df = data_df.dropna(subset=['comment', 'label']).drop_duplicates(data_df)

In [21]:
len(data_df)

209240

In [46]:
len(data_df[data_df['label'] == 1])

19715

In [47]:
len(data_df[data_df['label'] == 2])

23238

In [18]:
len(data_df[data_df['label'] == 3])

53000

In [19]:
len(data_df[data_df['label'] == 4])

67156

In [20]:
len(data_df[data_df['label'] == 5])

46131

In [21]:
max([len(c) for c in data_df['comment']])

2820

In [42]:
new_tf_path = './tf20190916.text'

In [48]:
def get_new_tf():
    words_tf = defaultdict(int)
    def words_gen(path):
        for line in open(path,'r',encoding = 'UTF-8'):
            yield line 
        return line
    for fname in os.listdir(new_tf_dir):            
        fpath = os.path.join(new_tf_dir, fname)
        for i ,line in enumerate(words_gen(fpath)):
            for w in line.strip().split(' '):
                #print('\r Current line: {}'.format(i),end = '')
                words_tf[w] += 1 
    with open(new_tf_path,'w',encoding = 'UTF-8') as f:
        f.writelines([' '.join([w,str(f)])+'\n' for w,f in words_tf.items()])
    print('New words TF updated!!Length:',len(words_tf))

In [49]:
get_new_tf()

New words TF updated!!Length: 3865604


# SIF embedding for Comment data

In [22]:
wiki_wv_path = './wv20190916.text'

In [23]:
wiki_tf_path = './tf20190916.text'

In [24]:
weightpara = 1e-4
rmpc = 1
(words, We) = data_io.getWordmap(wiki_wv_path)
word2weight = data_io.getWordWeight(wiki_tf_path, weightpara)

['1125723']


In [25]:
def sif_embedding(sentences):
    sents_segs = [' '.join(cut(sent)) for sent in sentences]
    x , m = data_io.sentences2idx(sents_segs, words)
    weight4ind = data_io.getWeight(words, word2weight)
    w = data_io.seq2weight(x, m, weight4ind) # get word weights
    param = params.params()
    #params = params()
    #params.rmpc = rmpc
    param.rmpc = rmpc
    #embedding = SIF_embedding.SIF_embedding(We, x, w, params)
    embedding = SIF_embedding.SIF_embedding(We, x, w, param)
    return embedding

In [47]:
def data_porc(comments_class,labels_class,scheme='SIF'):
    training_size = 10000
    testing_size = 2000
    shuffled_dataset = None
    shuffled_labels = None
    train_data = []
    train_labels = []
    test_data = []
    test_labels = []
    for i,(comments,labels) in enumerate(zip(comments_class,labels_class)):
        print('\r Processing volume:{}/{}'.format(i+1,len(comments_class)),end='')
        #embeddings = sif_embedding(comments)
        permutation = np.random.permutation(labels.shape[0])
        shuffled_dataset = embeddings[permutation,:]
        shuffled_labels = labels[permutation]
        train_data += [shuffled_dataset[:training_size,:]]
        train_labels += [shuffled_labels[:training_size].reshape(-1,1)]
        test_data += [shuffled_dataset[training_size:training_size+testing_size,:]]
        test_labels += [shuffled_labels[training_size:training_size+testing_size].reshape(-1,1)] 
    print()
    train_data, train_labels, test_data, test_labels = np.vstack(train_data), np.vstack(train_labels).reshape(-1), np.vstack(test_data), np.vstack(test_labels).reshape(-1)
    print('train_data shape:',train_data.shape)
    print('train_labels shape:',train_labels.shape)
    print('test_data shape:',test_data.shape)
    print('test_labels shape:',test_labels.shape)    
    p1 = np.random.permutation(train_labels.shape[0])    
    p2 = np.random.permutation(test_labels.shape[0])
    train_data ,train_labels, test_data, test_labels = train_data[p1,:], train_labels[p1], test_data[p2,:], test_labels[p2]
    print('Finish shuffling for dataset')
    return train_data, train_labels, test_data, test_labels 
    

In [48]:
comment_1 = data_df['comment'][data_df['label'] == 1]
labels_1 = np.array(data_df['label'][data_df['label'] == 1])
comment_2 = data_df['comment'][data_df['label'] == 2]
labels_2 = np.array(data_df['label'][data_df['label'] == 2])
comment_3 = data_df['comment'][data_df['label'] == 3]
labels_3 = np.array(data_df['label'][data_df['label'] == 3])
comment_4 = data_df['comment'][data_df['label'] == 4]
labels_4 = np.array(data_df['label'][data_df['label'] == 4])
comment_5 = data_df['comment'][data_df['label'] == 5]
labels_5 = np.array(data_df['label'][data_df['label'] == 5])
comments_class = [comment_1,comment_2,comment_3,comment_4,comment_5]
labels_class = [labels_1,labels_2,labels_3,labels_4,labels_5]

In [50]:
train_data, train_labels_raw, test_data, test_labels_raw = data_porc(comments_class,labels_class)

 Processing volume:5/5
train_data shape: (50000, 200)
train_labels shape: (50000,)
test_data shape: (10000, 200)
test_labels shape: (10000,)
Finish shuffling for dataset


# Save the embedding data

In [15]:
pickle_file = './embedding.pkl'

In [16]:
pickle_data = sif_embedding(data_df['comment'])
with open(pickle_file, 'wb') as f:
    pickle.dump(pickle_data, f, pickle.HIGHEST_PROTOCOL)

NameError: name 'sif_embedding' is not defined

In [17]:
pickle_data = None
with open(pickle_file, 'rb') as f:
    pickle_data = pickle.load(f)

In [18]:
pickle_data.shape

(209240, 200)

# Use K_Means to find best K value

In [7]:
def get_best_k_means(data):

    n_clusters_end = 10

    n_clusters_start = 2
    
    clusters = []
    
    score = 0
    
    K = 2
    
    while n_clusters_start <= n_clusters_end:
        
        km = KMeans(n_clusters=n_clusters_start)

        km.fit(data)  
        
        new_score = silhouette_score(X=data,labels=km.labels_.tolist())
        
        if new_score > score:
            
            score = new_score
            
            K = n_clusters_start
            
            clusters = km.labels_.tolist()
        
        print('Current  K: {} HighScore: {} CurrentScore: {} TryN: {}'.format(K,score,new_score,n_clusters_start))

        n_clusters_start += 1           
    
    return clusters

In [8]:
cluster = get_best_k_means(pickle_data)

Current  K: 2 HighScore: 0.037118787145977036 CurrentScore: 0.037118787145977036 TryN: 2
Current  K: 2 HighScore: 0.037118787145977036 CurrentScore: 0.010799116860554618 TryN: 3
Current  K: 2 HighScore: 0.037118787145977036 CurrentScore: 0.006993487743455404 TryN: 4
Current  K: 2 HighScore: 0.037118787145977036 CurrentScore: -0.004445469932645989 TryN: 5
Current  K: 2 HighScore: 0.037118787145977036 CurrentScore: -0.02071506215447534 TryN: 6
Current  K: 2 HighScore: 0.037118787145977036 CurrentScore: -0.02537705780011468 TryN: 7
Current  K: 2 HighScore: 0.037118787145977036 CurrentScore: -0.03843335260896459 TryN: 8
Current  K: 2 HighScore: 0.037118787145977036 CurrentScore: -0.03850687823909406 TryN: 9
Current  K: 2 HighScore: 0.037118787145977036 CurrentScore: -0.03172371409309053 TryN: 10


In [59]:
def get_cluster(data):
    
    km = KMeans(n_clusters=2)
    
    km.fit(data)  
    
    clusters = km.labels_.tolist()
    
    score = silhouette_score(X=data,labels=clusters)  
    
    print('Score: {} TryN: {}'.format(score,2))
    
    return clusters

In [60]:
cluster = get_cluster(pickle_data)

Score: 0.036395797764632795 TryN: 2


In [61]:
len(cluster)

209240

In [64]:
idx0 = [i for i,c in enumerate(cluster) if c == 0]
idx1 = [i for i,c in enumerate(cluster) if c == 1]

In [66]:
len(idx0),len(idx1),len(idx0)+len(idx1) == len(pickle_data)

(82383, 126857, True)

# K_Means Simply classify the comment into positive and negative 

In [71]:
[data_df['comment'].iloc[i] for i in idx0[:10]]

['三星半实打实的7分第一集在爱国主旋律内部做着各种置换与较劲但第二集才真正显露吴京的野心他终于抛弃李忠志了新增外来班底让硬件实力有机会和国际接轨开篇水下长镜头和诸如铁丝网拦截弹头的细节设计都让国产动作片重新封顶在理念上它甚至做到绣春刀2最想做到的那部分',
 '开篇长镜头惊险大气引人入胜结合了水平不俗的快剪下实打实的真刀真枪让人不禁热血沸腾特别弹簧床架挡炸弹空手接碎玻璃弹匣割喉等帅得飞起就算前半段铺垫节奏散漫主角光环开太大等也不怕作为一个中国人两个小时弥漫着中国强大得不可侵犯的氛围还是让那颗民族自豪心砰砰砰跳个不停',
 '15100吴京的冷峰在这部里即像成龙又像杰森斯坦森但体制外的同类型电影主角总是代表个人无能的政府需要求助于这些英雄才能解决难题体现的是个人的价值所以主旋律照抄这种模式实际上是有问题的我们以前嘲笑个人英雄主义却没想到捆绑爱国主义的全能战士更加难以下咽',
 '有几处情节设置过于尴尬彰显国家自豪感的部分稍显突兀',
 '符合有钱了续集反而拍更差这一放之四海而皆准的规律场面越做越大然而伴随着各种动作场面和特效场面的升级这一部的叙事反而变得非常凌乱格局颇大想拍成黑鹰坠落结果撑死最多也只是官方主旋律版的敢死队吴京确实有野心但论自我角色定位能力远不如同是动作演员出身的甄子丹',
 '战狼2的动作场景和战斗装备全线升级热血的打斗动作从头打到尾战狼2游走在电影审查红线的边界和政治安全的缝隙是部延续了第一部极具煽动爱国情绪的国产动作大片如此制作精良的影片还请多来一点',
 '火爆的场面拍出了好莱坞大片的感觉本片必将燃爆暑期',
 '血战钢锯岭中国人也会觉得好看因为它歌颂的宗教情怀是超越政权的但当你只想歌颂一个政权时很明显就低了一个层次甚至充满了现实乃至投机的考量高下立见',
 '翻了一下我给第一部的评价是四星当时觉得挺燃的这部其实在完成度上更接近好莱坞的制作了每个步骤每个人物的走向都很顺滑没有任何出人意料的地方只给三星是因为看看最近现实世界的一切抱歉我在影院里燃不起来只是觉得一切都很魔幻当然开头的强拆是最有现实感的一幕了',
 '作为主旋律影片为啥用奇异恩典配乐画内镜头还是中国军人']

In [72]:
[data_df['comment'].iloc[i] for i in idx1[:10]]

['吴京意淫到了脑残的地步看了恶心想吐',
 '首映礼看的太恐怖了这个电影不讲道理的完全就是吴京在实现他这个小粉红的英雄梦各种装备轮番上场视物理逻辑于不顾不得不说有钱真好随意胡闹',
 '吴京的炒作水平不输冯小刚但小刚至少不会用主旋律来炒作吴京让人看了不舒服为了主旋律而主旋律为了煽情而煽情让人觉得他是个大做作大谎言家729更新片子整体不如湄公河行动1整体不够流畅编剧有毒台词尴尬2刻意做作的主旋律煽情显得如此不合时宜而又多余',
 '凭良心说好看到不像战狼1的续集完虐湄公河行动',
 '中二得很',
 '犯我中华者虽远必诛吴京比这句话还要意淫一百倍',
 '脑子是个好东西希望编剧们都能有',
 '犯我中华者虽远必诛是有多无脑才信这句话',
 '这部戏让人看的热血沸腾对吴京路转粉最后的彩蛋让我们没有理由不期待下一部',
 '假嗨特别恶心的电影']

# Generating training and testing datasets

In [73]:
dataset_0 = pickle_data[idx0,:]
labels_0 = np.array([0 for __ in idx0])
dataset_1 = pickle_data[idx1,:]
labels_1 = np.array([0 for __ in idx1])

In [74]:
dataset_0.shape,labels_0.shape,dataset_1.shape,labels_1.shape

((82383, 200), (82383,), (126857, 200), (126857,))

In [75]:
datasets_class = [dataset_0,dataset_1]
labels_class = [labels_0,labels_1]

In [78]:
def dataset_porc(datasets_class,labels_class):
    training_size = 75000
    testing_size = 5000
    shuffled_dataset = None
    shuffled_labels = None
    train_data = []
    train_labels = []
    test_data = []
    test_labels = []
    for i,(dataset,labels) in enumerate(zip(datasets_class,labels_class)):
        print('\r Processing volume:{}/{}'.format(i+1,len(datasets_class)),end='')
        #embeddings = sif_embedding(comments)
        permutation = np.random.permutation(labels.shape[0])
        shuffled_dataset = dataset[permutation,:]
        shuffled_labels = labels[permutation]
        train_data += [shuffled_dataset[:training_size,:]]
        train_labels += [shuffled_labels[:training_size].reshape(-1,1)]
        test_data += [shuffled_dataset[training_size:training_size+testing_size,:]]
        test_labels += [shuffled_labels[training_size:training_size+testing_size].reshape(-1,1)] 
    print()
    train_data, train_labels, test_data, test_labels = np.vstack(train_data), np.vstack(train_labels).reshape(-1), np.vstack(test_data), np.vstack(test_labels).reshape(-1)
    print('train_data shape:',train_data.shape)
    print('train_labels shape:',train_labels.shape)
    print('test_data shape:',test_data.shape)
    print('test_labels shape:',test_labels.shape)    
    p1 = np.random.permutation(train_labels.shape[0])    
    p2 = np.random.permutation(test_labels.shape[0])
    train_data ,train_labels, test_data, test_labels = train_data[p1,:], train_labels[p1], test_data[p2,:], test_labels[p2]
    print('Finish shuffling for dataset')
    return train_data, train_labels, test_data, test_labels 
    

In [79]:
train_data, train_labels_raw, test_data, test_labels_raw = dataset_porc(datasets_class,labels_class)

 Processing volume:2/2
train_data shape: (150000, 200)
train_labels shape: (150000,)
test_data shape: (10000, 200)
test_labels shape: (10000,)
Finish shuffling for dataset


In [80]:
num_labels = 5
ebd_size = 200
def reformat(dataset, labels):
    dataset = dataset.reshape((-1, ebd_size)).astype(np.float32)
    # Map 0 to [1.0, 0.0, 0.0 ...], 1 to [0.0, 1.0, 0.0 ...]
    labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
    return dataset, labels
train_data, train_labels = reformat(train_data, train_labels_raw)
test_data, test_labels = reformat(test_data, test_labels_raw)
print('Training set', train_data.shape, train_labels.shape)
print('Test set', test_data.shape, test_labels.shape)

Training set (150000, 200) (150000, 5)
Test set (10000, 200) (10000, 5)


# Use full connection NN training and predicting

In [81]:
def accuracy(predictions, labels):
    return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))/ predictions.shape[0])

In [82]:
batch_size = 200
n_hiddden = 1024
graph = tf.Graph()
with graph.as_default():

    # Input data. For the training data, we use a placeholder that will be fed
    # at run time with a training minibatch.
    tf_train_dataset = tf.placeholder(tf.float32,shape=(batch_size, ebd_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_test_dataset = tf.constant(test_data)
  
    # Variables.
    weights1 = tf.Variable(tf.truncated_normal([ebd_size, n_hiddden]))
    biases1 = tf.Variable(tf.zeros([n_hiddden]))
    weights2 = tf.Variable(tf.truncated_normal([n_hiddden, num_labels]))
    biases2 = tf.Variable(tf.zeros([num_labels]))
    
    # Training computation.
    hidden1 = tf.nn.relu(tf.matmul(tf_train_dataset, weights1) + biases1)

    logits = tf.matmul(hidden1, weights2) + biases2
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
    
    # Optimizer.
    optimizer = tf.train.GradientDescentOptimizer(0.001).minimize(loss)
  
    hidden1_test  = tf.nn.relu(tf.matmul(tf_test_dataset, weights1) + biases1)
    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)
    test_prediction = tf.nn.softmax(tf.matmul(hidden1_test, weights2) + biases2)

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See @{tf.nn.softmax_cross_entropy_with_logits_v2}.



In [83]:
num_steps = 5001

with tf.Session(graph=graph) as session:
    tf.global_variables_initializer().run()
    print("Initialized")
    for step in range(num_steps):
        # Pick an offset within the training data, which has been randomized.
        # Note: we could use better randomization across epochs.
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        # Generate a minibatch.
        batch_data = train_data[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        # Prepare a dictionary telling the session where to feed the minibatch.
        # The key of the dictionary is the placeholder node of the graph to be fed,
        # and the value is the numpy array to feed to it.
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 500 == 0):
            print("Minibatch loss at step %d: %f" % (step, l))
            print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
    print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 140.672272
Minibatch accuracy: 9.5%
Minibatch loss at step 500: 0.201113
Minibatch accuracy: 99.5%
Minibatch loss at step 1000: 0.041698
Minibatch accuracy: 99.5%
Minibatch loss at step 1500: 0.000110
Minibatch accuracy: 100.0%
Minibatch loss at step 2000: 0.058994
Minibatch accuracy: 99.5%
Minibatch loss at step 2500: 0.000000
Minibatch accuracy: 100.0%
Minibatch loss at step 3000: 0.141915
Minibatch accuracy: 99.5%
Minibatch loss at step 3500: 0.000000
Minibatch accuracy: 100.0%
Minibatch loss at step 4000: 0.203091
Minibatch accuracy: 99.0%
Minibatch loss at step 4500: 0.000000
Minibatch accuracy: 100.0%
Minibatch loss at step 5000: 0.000000
Minibatch accuracy: 100.0%
Test accuracy: 99.9%
